# Assignment 1C - Question 2
## Semantic Person Search

In [17]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import keras
from keras import layers


import os
import datetime
import numpy

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorboard import notebook
from tensorflow.keras.preprocessing.image import Iterator

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import pydot
import IPython
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model
import imageio
import cv2

In [2]:
train = pd.read_csv('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Train.csv')
test = pd.read_csv('CAB420_Assessment_1C_Data/Data/Q2/Q2/Test_Data/Test.csv')

In [20]:
# train_img = []
# gnd_img = []
# files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Originals/*.png')
# for myfile in files:
#     image = cv2.imread(myfile, 0)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     gnd_img.append(myfile[58:])
#     train_img.append(image)

# train_img = np.asarray(train_img)
# gnd_img = np.array(gnd_img)

# print('Training shape: ', train_img.shape)
# print('Training gnd: ', gnd_img[1:10])





train = []
gnd = []
files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Originals/*.png')
for myfile in files:
    im = imageio.imread(myfile)
    print(im.shape)
    gnd.append(myfile[58:])
    train.append(im)

train = np.array(train)
gnd = np.array(gnd)

print('Training shape: ', train.shape)
print('Training gnd: ', gnd[1:10])

(226, 144, 3)
(207, 114, 3)
(220, 127, 3)
(188, 145, 3)
(249, 135, 3)
(193, 163, 3)
(275, 122, 3)
(145, 74, 3)
(206, 143, 3)
(393, 186, 3)
(231, 128, 3)
(155, 89, 3)
(189, 98, 3)
(190, 94, 3)
(216, 126, 3)
(202, 98, 3)
(171, 87, 3)
(233, 120, 3)
(189, 109, 3)
(172, 96, 3)
(157, 136, 3)
(187, 112, 3)
(257, 135, 3)
(204, 100, 3)
(225, 118, 3)
(167, 78, 3)
(180, 118, 3)
(191, 101, 3)
(240, 131, 3)
(214, 106, 3)
(209, 108, 3)
(199, 99, 3)
(191, 91, 3)
(205, 119, 3)
(145, 92, 3)
(219, 113, 3)
(179, 150, 3)
(218, 125, 3)
(188, 99, 3)
(205, 94, 3)
(210, 155, 3)
(179, 100, 3)
(188, 95, 3)
(211, 140, 3)
(197, 106, 3)
(181, 91, 3)
(236, 128, 3)
(186, 128, 3)
(166, 89, 3)
(218, 123, 3)
(150, 70, 3)
(187, 72, 3)
(182, 98, 3)
(166, 91, 3)
(229, 127, 3)
(204, 104, 3)
(157, 60, 3)
(250, 141, 3)
(216, 134, 3)
(239, 134, 3)
(212, 102, 3)
(179, 113, 3)
(187, 113, 3)
(295, 153, 3)
(159, 74, 3)
(188, 142, 3)
(187, 93, 3)
(213, 129, 3)
(188, 89, 3)
(249, 149, 3)
(199, 140, 3)
(273, 127, 3)
(185, 86, 3)
(18

In [4]:
test_originals = []
gnd_test = []
files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Test_Data/Originals/*.png')
for myfile in files:
    image = cv2.imread(myfile)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gnd_test.append(myfile[57:])
    test_originals.append(image)

test_originals = np.asarray(test_originals)
gnd_test = np.array(gnd_test)
#print('Testing shape: ', test_originals.shape)
#print('Testing gnd: ', gnd_test[1:10])

In [5]:
train = train.drop(columns=['torcol2','torcol3','tortex','torcol3','legcol2','legcol3','legtex','pose'])


train_gender = train.iloc[:,1]
train_gender = np.asarray(train_gender)
train_tortyp = train.iloc[:,2]
train_tortyp = np.asarray(train_tortyp)
train_torcol = train.iloc[:,3]
train_torcol = np.asarray(train_torcol)
train_legtyp = train.iloc[:,4]
train_legtyp = np.asarray(train_legtyp)
train_legcol = train.iloc[:,5]
train_legcol = np.asarray(train_legcol)
train_luggage = train.iloc[:,6] 
train_luggage = np.asarray(train_luggage)


In [6]:
test = test.drop(columns=['torcol2','torcol3','tortex','torcol3','legcol2','legcol3','legtex','pose'])

test_gender = test.iloc[:,1]
test_gender = np.asarray(test_gender)
test_tortyp = test.iloc[:,2]
test_tortyp = np.asarray(test_tortyp)
test_torcol = test.iloc[:,3]
test_torcol = np.asarray(test_torcol)
test_legtyp = test.iloc[:,4]
test_legtyp = np.asarray(test_legtyp)
test_legcol = test.iloc[:,5]
test_legcol = np.asarray(test_legcol)
test_luggage = test.iloc[:,6] 
test_luggage = np.asarray(test_luggage)

In [7]:
#fig = plt.figure(figsize=[20, 20])
#for i in range(100):
#    ax = fig.add_subplot(10, 10, i + 1)
#    ax.imshow(test_originals[i])

In [8]:
#,tortyp,torcol,legtyp,legcol,luggage]
#, train_tortyp,train_torcol,train_legtyp,train_legcol,train_luggage]
#, test_tortyp, test_torcol, test_legtyp, test_legcol, test_luggage]

In [9]:
# inputs = keras.Input(shape=(144, 226, 3, ), name='img')
# x = layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu')(inputs)
# x = layers.MaxPool2D(pool_size=(2, 2))(x)
# x = layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(x)
# x = layers.MaxPool2D(pool_size=(2, 2))(x)
# x = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
# x = layers.Flatten()(x)
# x1 = layers.Dense(64, activation='relu')(x)
# gender = layers.Dense(3, name='gender_out')(x1)

# # x2 = layers.Dense(64, activation='relu')(x)
# # tortyp = layers.Dense(3, name='tortyp_out')(x2)

# # x3 = layers.Dense(64, activation='relu')(x)
# # torcol = layers.Dense(11, name='torcol_out')(x3)

# # x4 = layers.Dense(64, activation='relu')(x)
# # legtyp = layers.Dense(3, name='legtyp_out')(x4)

# # x5 = layers.Dense(64, activation='relu')(x)
# # legcol = layers.Dense(11, name='legocol_out')(x5)

# # x6 = layers.Dense(64, activation='relu')(x)
# # luggage = layers.Dense(3, name='luggage_out')(x6)

# model_cnn = keras.Model(inputs=inputs, outputs=gender, name='A1CQ2')


inputs = keras.Input(shape=(144, 226, 3, ), name='img')
x = layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='same')(inputs)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(11, activation='softmax')(x)

model_cnn = keras.Model(inputs=inputs, outputs=outputs, name='SVHN_CNN_Model')


model_cnn.summary()

Model: "SVHN_CNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 144, 226, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 144, 226, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 72, 113, 8)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 113, 16)       1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 56, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 56, 32)        4640      
_________________________________________________________________
flatten (Flatten)            (None, 64512)          

In [10]:
model_cnn.compile(loss=['mean_squared_error', keras.losses.SparseCategoricalCrossentropy(from_logits=True)],
              optimizer=keras.optimizers.RMSprop())

In [11]:
history = model_cnn.fit(train_img, train_gender,
                    batch_size=64,
                    epochs=20,
                    validation_data=(test_originals, test_gender))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).